<a href="https://colab.research.google.com/github/tayfununal/Normalizing-Flows/blob/main/ode_new_with_mse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ode solver için gerekli kütüphaneler
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

# Deep learning için gerekli kütüphaneler 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

plt.style.use('seaborn-poster')

%matplotlib inline
"""
def F(t,s):       # Diff eq F(t,s) = 0
  return [np.cos(t), -np.sin(t)]
"""
tf.random.set_seed(39)
np.random.seed(39)

initial_state = [0., 1.]                  # t0 anındaki başlangıç değerleri
t_starting_and_last = [0, 2*np.pi]        # Fonksiyonun tanımlı olacağı alanın başlangıç ve bitiş noktasını tanımlıyoruz.
t_eval = np.arange(0, 2*np.pi, 0.01)      # t_starting_and_last ile belirlediğimiz aralıkta hesaplanacak t değerlerini bu şekilde girebiliriz, hiçbirşey girilmez ise default olarak kendisi parçalıyor.

def F(t,y):       # Diff eq F(t,s) = 0
  return [y[1], -y[0]]

sol = solve_ivp(fun= F, t_span= t_starting_and_last, y0= initial_state, t_eval= t_eval, dense_output=True)


plt.figure(figsize = (8, 8))

plt.subplot(221)
plt.plot(sol.t, sol.y[0])
plt.xlabel('t')
plt.ylabel('H[0] = sin(t)')

plt.subplot(222)
plt.plot(sol.t, sol.y[0] - np.sin(sol.t))
plt.xlabel('t')
plt.ylabel('H[0] - sin(t)')

plt.subplot(223)
plt.plot(sol.t, sol.y[1])
plt.xlabel('t')
plt.ylabel('H[1] = cos(t)')

plt.subplot(224)
plt.plot(sol.t, sol.y[1] - np.cos(sol.t))
plt.xlabel('t')
plt.ylabel('H[1] - cos(t)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (4, 4))
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()


In [ ]:
y = sol.y.T # etiketlerimiz
t = sol.t   # t değerlerimiz

In [ ]:
t = t.reshape(([t.shape[0],1]))
shuffle = np.concatenate((y,t), axis=1)
np.random.shuffle(shuffle)
shuffle

In [ ]:
y = shuffle[:,:2]
t = shuffle[:,2]
print('y:',y, '\nt:',t)

In [ ]:
# Hamiltonian for initial values [0,1]
h_0 = H(np.array([initial_state], dtype=np.float32))

def H(z):
  return tf.reduce_sum((z**2)/2,axis=1)

mse = tf.keras.losses.MSE
def custom_loss(y_true, y_pred):
  return mse(y_true, y_pred) + (2/629)*(h_0 - H(y_pred))**2

In [ ]:
# Model
girdi = Input(shape=(1,))

hidden = Dense(512, activation= 'tanh')(girdi)
hidden = Dense(512, activation= 'tanh')(hidden)

hidden = Dense(2)(hidden)

model = Model(inputs=girdi, outputs=hidden)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=opt, loss= custom_loss)
model.fit(x=t, y= y, epochs=1000, batch_size=64)

In [ ]:
t_test = np.arange(0, 2*np.pi, 0.02).reshape((315,1)) #629
pred = model.predict(t_test)

hamiltonian = H(pred)
true_hamiltonian = H(sol.y.T)

plt.figure(figsize = (10, 5))

plt.subplot(121)
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.plot(pred[:, 0], pred[:, 1])
plt.xlabel('q')
plt.ylabel('p')

plt.subplot(122)
plt.plot(t_test,hamiltonian)
plt.plot(t_eval, true_hamiltonian)
plt.xlabel('t')
plt.ylabel('H')

plt.tight_layout()
plt.show()

In [ ]:
hamiltonian

In [ ]:
array = np.array([0.507433  , 0.504659  , 0.50216323, 0.49991852, 0.4980037 ,
       0.49644467, 0.4952755 , 0.49449575, 0.49410582, 0.49410534,
       0.49449462, 0.49516642, 0.4955425 , 0.4951452 , 0.4951563 ,
       0.4955759 , 0.49640396, 0.4976414 , 0.49906453, 0.5008722 ,
       0.503089  , 0.50576836, 0.5046942 , 0.5022956 , 0.5001174 ,
       0.49815154, 0.49658126, 0.49540618, 0.49462676, 0.49424288,
       0.4942544 , 0.49466112, 0.49546337, 0.49653515, 0.49775976,
       0.49940458, 0.50131154, 0.50055635, 0.49889642, 0.49753362,
       0.49648997, 0.49584055, 0.4955849 , 0.4957233 , 0.49612474,
       0.49628812, 0.4966917 , 0.49716294, 0.4980048 , 0.49927992,
       0.49979955, 0.49823853, 0.49705625, 0.49625856, 0.49584517,
       0.49581647, 0.49617198, 0.4967555 , 0.49757537, 0.49846375,
       0.49844468, 0.49744564, 0.49681395, 0.49650937, 0.49653125,
       0.4965572 , 0.49688858, 0.49764863, 0.49882257, 0.4984637 ,
       0.49767068, 0.49667823, 0.49608782, 0.49589524, 0.49609208,
       0.49654788, 0.49711493, 0.4979014 , 0.4975925 , 0.49751362,
       0.49775818, 0.49841943, 0.49872875, 0.49760437, 0.49677208,
       0.49634346, 0.49630716, 0.49667412, 0.49698797, 0.4975296 ,
       0.4982952 , 0.49774176, 0.4971248 , 0.49689206, 0.49703065,
       0.49746823, 0.49830467, 0.49795473, 0.4975345 , 0.497414  ,
       0.4976974 , 0.4983784 , 0.4994502 , 0.499205  , 0.49844784,
       0.49787468, 0.4976682 , 0.4978489 , 0.4981966 , 0.4982372 ,
       0.49866593, 0.49949712, 0.49913615, 0.4985578 , 0.49834442,
       0.49837923, 0.49882072, 0.49966905, 0.49900305, 0.49856615,
       0.49844658, 0.49846053, 0.49882603, 0.4995399 , 0.49898314,
       0.49862942, 0.49866825, 0.49905622, 0.4995523 , 0.49927562,
       0.49876308, 0.4986466 , 0.49880797, 0.49852514, 0.49849373,
       0.49882758, 0.49956423, 0.5004488 , 0.49974597, 0.49922448,
       0.4990727 , 0.49923843, 0.4997518 , 0.5003339 , 0.49994376,
       0.49960124, 0.499587  , 0.49980813, 0.5003871 , 0.5003537 ,
       0.50037384, 0.50073737, 0.5006107 , 0.5007889 , 0.500791  ,
       0.5008527 , 0.50116444, 0.50074863, 0.5006482 , 0.50088006,
       0.50127226, 0.5010515 , 0.5011072 , 0.5008746 , 0.501014  ,
       0.50147617, 0.5016089 , 0.501818  , 0.5021543 , 0.5018649 ,
       0.50138247, 0.5011778 , 0.5011533 , 0.50100607, 0.5011956 ,
       0.5014507 , 0.50133747, 0.5015927 , 0.50193316, 0.50176674,
       0.5014296 , 0.50134444, 0.5014743 , 0.5009831 , 0.50076467,
       0.5009246 , 0.5014489 , 0.501599  , 0.50158954, 0.5017065 ,
       0.5019766 , 0.50171036, 0.50148   , 0.5015337 , 0.5013685 ,
       0.5011338 , 0.50128263, 0.5017135 , 0.5020098 , 0.50147396,
       0.5011275 , 0.5010035 , 0.5012097 , 0.50183123, 0.501642  ,
       0.5013819 , 0.5012484 , 0.5013516 , 0.50186694, 0.50127876,
       0.5006291 , 0.50026864, 0.5000725 , 0.5001841 , 0.500701  ,
       0.5009425 , 0.5004943 , 0.5001997 , 0.49989533, 0.4995483 ,
       0.49889126, 0.49862802, 0.49869904, 0.49915102, 0.50000846,
       0.5010547 , 0.50022733, 0.49920586, 0.49858427, 0.49834004,
       0.49830565, 0.49869883, 0.49887168, 0.49836358, 0.49789143,
       0.49747694, 0.49740988, 0.49772212, 0.4977911 , 0.4979282 ,
       0.49839616, 0.498568  , 0.49779597, 0.497295  , 0.49713847,
       0.49687326, 0.49683106, 0.49691746, 0.497351  , 0.4975621 ,
       0.4976483 , 0.49815863, 0.4977097 , 0.49692568, 0.49639142,
       0.49598852, 0.495646  , 0.49527234, 0.4948752 , 0.49468368,
       0.49483532, 0.4950747 , 0.49573717, 0.4961167 , 0.49542654,
       0.49447   , 0.49374676, 0.49334097, 0.49320883, 0.49346605,
       0.4939127 , 0.4933956 , 0.49327245, 0.49333018, 0.4929064 ,
       0.49288428, 0.49326378, 0.49331462, 0.49328038, 0.4927141 ,
       0.4920911 , 0.49184534, 0.49169922, 0.4918204 , 0.49234724,
       0.49328157, 0.49299133, 0.4919495 , 0.49120894, 0.49075323,
       0.49061492, 0.49080527, 0.49137706, 0.4923404 , 0.49369505,
       0.49190828, 0.4898859 , 0.48796383, 0.48623988, 0.48488456,
       0.4839356 , 0.4833455 , 0.48312807, 0.483306  , 0.48387945,
       0.4848484 , 0.4862106 , 0.4878685 , 0.48990768, 0.49232835,
       0.4951301 , 0.49831355, 0.501878  , 0.505824  , 0.51015145])
plt.figure(figsize = (4, 4))
plt.plot(t_test,array,c='red')
plt.plot(t_test, hamiltonian, c='green')
plt.plot(t_test, [0.5 for i in range(t_test.shape[0])])

plt.xlabel('t')
plt.ylabel('H')
plt.show()